In [1]:
# ------------------------- 필요한 라이브러리 불러오기 --------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pickle

In [2]:
import numpy as np
import random
import os

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)


In [3]:
# ------------------------- 데이터셋 불러오기 --------------------------------------------
# pickle로 저장된 이미지 데이터셋을 불러옵니다
# 데이터는 'image'와 'label' 키를 가진 딕셔너리 구조입니다
save_path = "../data/gtsrb_train_dataset_2000_balanced.pkl"
with open(save_path, "rb") as f:
    dataset = pickle.load(f)

images = dataset['image']    # PIL.Image 객체의 리스트
labels = dataset['label']    # 클래스 번호 리스트 (0~42)

In [4]:
# ------------------------- 이미지 전처리 --------------------------------------------
# 이미지 전처리 함수 정의
# PIL 이미지를 224x224로 리사이즈 후 numpy 배열로 변환하고 정규화
def preprocess(img):
    img = img.resize((224, 224))
    arr = np.array(img).astype(np.float32) / 255.0  # 0~1 사이 정규화
    # -1 ~ 1 범위로 다시 스케일링 (평균 0.5, std 0.5 기준)
    arr = (arr - 0.5) / 0.5
    return arr

In [5]:
# 전체 이미지에 대해 전처리 적용
X = np.array([preprocess(img) for img in images])
y = np.array(labels)

# 이미지와 라벨 순서 섞기
X, y = shuffle(X, y, random_state=seed)

# 훈련/테스트 데이터 분할
#    test_size=0.2 → 80% 훈련, 20% 테스트
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed, stratify=y
)

In [8]:
# ------------------------- Keras CNN 모델 정의 --------------------------------------------
# Sequential 모델을 사용해 CNN 구성
model = Sequential([
    # 첫 번째 합성곱 층: 입력 224x224x3 → 출력 112x112x32 (MaxPooling으로 절반)
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # 두 번째 합성곱 층: 출력 56x56x64
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # 세 번째 합성곱 층: 출력 28x28x128
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # GlobalAveragePooling2D: 모든 공간 정보 평균 → (Batch, 128)
    GlobalAveragePooling2D(),

    # 완전 연결층
    Dense(256, activation='relu'),

    # 출력층: 43개의 교통 표지판 클래스에 대한 softmax 확률 출력
    Dense(43, activation='softmax')
])

c:\Users\SSAFY\Desktop\hw\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# ------------------------- 모델 컴파일 --------------------------------------------
# loss: sparse_categorical_crossentropy 사용 (정수형 라벨에 적합)
# optimizer: Adam 사용, 학습률 0.001로 설정
# metrics: 정확도 평가
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
# ------------------------- 모델 학습 --------------------------------------------
# 배치 크기 설정: 한 번에 8개 샘플 학습
# 에포크 수: 전체 데이터를 10회 반복
model.fit(X, y, batch_size=8, epochs=10, verbose=1)

Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.0399 - loss: 3.7008
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.0566 - loss: 3.3791
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.0834 - loss: 3.2485
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.1041 - loss: 3.1645
Epoch 5/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.1244 - loss: 3.0828
Epoch 6/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.1395 - loss: 3.0017
Epoch 7/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.1734 - loss: 2.9175
Epoch 8/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.1921 - loss: 2.8249
Epoch 9/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.2174 - loss: 2.7232
Epoch 10/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.2422 - loss: 2.6228


In [11]:
# ------------------------- 모델 평가 --------------------------------------------
# 테스트 평가: 정확도 및 손실 계산
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"\n테스트 손실: {loss:.4f}")
print(f"테스트 정확도: {accuracy*100:.2f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.2298 - loss: 2.5393

테스트 손실: 2.5393
테스트 정확도: 22.98%
